<a href="https://colab.research.google.com/github/Ishaan20072612/Rag-Architecture-md-Processing/blob/main/Rag_md_Reader_and_Embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install nbstripout

In [ ]:
pip install -qU langchain_community

***Step 1: Downloading The Required Libraries***

In [ ]:
import os
from langchain.document_loaders import TextLoader
from transformers import pipeline
from langchain_text_splitters import MarkdownHeaderTextSplitter
from sentence_transformers import SentenceTransformer
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings

***Step 2: Loading The File***

In [ ]:
loader = TextLoader("/content/tennis_details.md") # Copied the file path
text_doc = loader.load() # Loaded the required file
print(text_doc[0].page_content)

# Tennis

## Introduction
Tennis is a popular sport played between two players (singles) or two teams of two players each (doubles). The game involves using a racket to hit a ball over a net into the opponent's court.

## Basic Rules
- A match can be played as best of three or five sets.
- Each set consists of games, and each game consists of points.
- Points are scored as **0 (Love), 15, 30, 40**, and then **game**.
- A player must win a game by at least **two points**.
- The ball must land within the designated court boundaries.

## Scoring System
```plaintext
0 points  -> Love
1 point   -> 15
2 points  -> 30
3 points  -> 40
4 points  -> Game (if leading by 2)
Deuce     -> 40-40 (must win two consecutive points to win the game)
Advantage -> If a player wins a point at deuce, they gain the advantage
```

## Famous Tournaments
- **Grand Slam Events**:
  - Australian Open
  - French Open
  - Wimbledon
  - US Open

## Equipment
- **Racket**: Used to hit the ball.
- **Tennis Ball**: Yello

***Step 3: Splitting The Text***

In [ ]:
split_cond = [ ( "##" , "Title" ) ]
splitter = MarkdownHeaderTextSplitter(split_cond) #Splitting the text into the chunks
splitti = splitter.split_text(text_doc[0].page_content)
print(splitti)

[Document(metadata={}, page_content='# Tennis'), Document(metadata={'Title': 'Introduction'}, page_content="Tennis is a popular sport played between two players (singles) or two teams of two players each (doubles). The game involves using a racket to hit a ball over a net into the opponent's court."), Document(metadata={'Title': 'Basic Rules'}, page_content='- A match can be played as best of three or five sets.\n- Each set consists of games, and each game consists of points.\n- Points are scored as **0 (Love), 15, 30, 40**, and then **game**.\n- A player must win a game by at least **two points**.\n- The ball must land within the designated court boundaries.'), Document(metadata={'Title': 'Scoring System'}, page_content='```plaintext\n0 points  -> Love\n1 point   -> 15\n2 points  -> 30\n3 points  -> 40\n4 points  -> Game (if leading by 2)\nDeuce     -> 40-40 (must win two consecutive points to win the game)\nAdvantage -> If a player wins a point at deuce, they gain the advantage\n```'

In [ ]:
text_chunks = []
for i in splitti:
  text_chunks.append(i.page_content)
print(text_chunks)
len(text_chunks)

['# Tennis', "Tennis is a popular sport played between two players (singles) or two teams of two players each (doubles). The game involves using a racket to hit a ball over a net into the opponent's court.", '- A match can be played as best of three or five sets.\n- Each set consists of games, and each game consists of points.\n- Points are scored as **0 (Love), 15, 30, 40**, and then **game**.\n- A player must win a game by at least **two points**.\n- The ball must land within the designated court boundaries.', '```plaintext\n0 points  -> Love\n1 point   -> 15\n2 points  -> 30\n3 points  -> 40\n4 points  -> Game (if leading by 2)\nDeuce     -> 40-40 (must win two consecutive points to win the game)\nAdvantage -> If a player wins a point at deuce, they gain the advantage\n```', '- **Grand Slam Events**:\n- Australian Open\n- French Open\n- Wimbledon\n- US Open', '- **Racket**: Used to hit the ball.\n- **Tennis Ball**: Yellow-green in color, designed for optimal bounce.\n- **Court**: Ca

7

***Step 4: Creating Embeddings***

In [ ]:
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
def embed_chunk(chunk):
  return embedding_model.encode([chunk] , normalize_embeddings=True)
sample_embeddings = embed_chunk(text_chunks)
print(sample_embeddings)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

[[ 2.45774128e-02  1.48011735e-02  5.18463254e-02 -7.59667531e-02
  -9.86059010e-02  1.63239054e-02  8.79042298e-02  6.02516904e-02
   8.84635895e-02  1.29884765e-01 -5.67051694e-02  4.10786793e-02
  -2.85655190e-03  2.36859173e-02  1.62318926e-02 -3.24991979e-02
   2.05462445e-02 -2.89406776e-02  5.37383109e-02  1.23084104e-02
   3.42429020e-02 -4.62833121e-02  3.65829058e-02 -9.38248411e-02
  -1.99453868e-02  3.73758026e-04 -7.28232190e-02  6.46518394e-02
  -3.76451053e-02  4.21944074e-02 -5.50501347e-02  3.23233865e-02
  -2.31778212e-02  4.07123715e-02 -1.92247346e-01 -3.21713798e-02
  -4.56059910e-02  5.64202964e-02 -1.90197974e-02  4.14559878e-02
   4.61168550e-02 -5.83031923e-02  1.99592132e-02  6.55939803e-02
  -4.69151302e-04  1.11691490e-01 -4.80918996e-02  1.13276400e-01
   3.87044027e-02 -4.23418768e-02 -7.18034506e-02  4.09240611e-02
   4.01694626e-02 -4.08084132e-03  9.91938636e-02 -4.47080150e-04
   3.92496511e-02  1.89655889e-02  1.94423106e-02  1.09847318e-02
   7.41450

***Step 5: Storing into Vector DataBase***

In [ ]:
!pip install -qU chromadb
vector_db = Chroma.from_texts( text_chunks , HuggingFaceEmbeddings( model_name = "all-MiniLM-L6-v2" ) , persist_directory = "/tmp/chromadb/" )
vector_db._collection.get(include=["embeddings", "documents"])

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.6/19.6 MB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 73.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 89.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.0/120.0 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.6/201.6 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 5.4 MB/s eta

/tmp/ipython-input-1932018546.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  vector_db = Chroma.from_texts( text_chunks , HuggingFaceEmbeddings( model_name = "all-MiniLM-L6-v2" ) , persist_directory = "/tmp/chromadb/" )


{'ids': ['bb7ba563-f4ba-43c2-afa2-70354de9ef93',
  '10ea1a07-3025-415f-bafb-01898c472dd7',
  'dd9359ee-4f35-45b1-90a3-93765cc2d490',
  '6d1217ae-44c2-4ba9-a31c-66b251d60758',
  '31ce2dc7-1f17-4370-a77f-8affda9363a9',
  '5f12617d-f3b4-4a89-b9c0-8db30cc224c2',
  '41db7879-e6b2-426f-bf2f-5adf20dadeb9'],
 'embeddings': array([[ 0.0227568 ,  0.05737348,  0.06708645, ..., -0.09128203,
          0.03132669,  0.02229537],
        [ 0.0431284 ,  0.013731  ,  0.04093742, ..., -0.01512668,
         -0.00087326,  0.03250714],
        [ 0.03571488,  0.02733695, -0.01912352, ...,  0.03562166,
         -0.01786361,  0.02190564],
        ...,
        [ 0.01810161,  0.02553317,  0.02062308, ..., -0.07975577,
         -0.04529488,  0.0281473 ],
        [ 0.05494466,  0.04179733,  0.03473553, ..., -0.04557597,
          0.05043148,  0.04526827],
        [ 0.02283715,  0.03019896,  0.07116921, ..., -0.01202571,
         -0.01404738,  0.03577407]]),
 'documents': ['# Tennis',
  "Tennis is a popular sport p

***Step 6: Logging to HuggingFace Hub***

In [ ]:
from transformers import pipeline
from huggingface_hub import login
login()

***Step 7: Using the pipeline to generate the text***

In [ ]:
pipe = pipeline("text-generation" , model = "meta-llama/Llama-3.2-1B")
print(pipe)

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

Device set to use cpu


***Step 8: Setting up a Retrieval-based QA Chain***

In [ ]:
def retrieve_and_generate(query,threshold=1):
  search_results = vector_db.similarity_search_with_score(query,k=1)
  if not search_results or search_results[0][1] > threshold:
    return "I don't know the answer"
  retrieve_context = search_results[0][0].page_content
  similarity_score = search_results[0][1]
  print(f"Similarity Score: {similarity_score}")
  print(f"Retrieved Context: {retrieve_context}")
  prompt = f"Answer the question using the given context \n Context: {retrieve_context} \n Question: {query} \n Answer:"
  response = pipe(prompt , max_new_tokens = 100)
  print(response)
  return response[0]["generated_text"]

***Step 9: Asking a question and retrieving the answer***

In [ ]:
ques = "What is tennis?"
response = retrieve_and_generate(ques)
print(response)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Similarity Score: 0.29123160243034363
Retrieved Context: Tennis is a popular sport played between two players (singles) or two teams of two players each (doubles). The game involves using a racket to hit a ball over a net into the opponent's court.
[{'generated_text': "Answer the question using the given context \n Context: Tennis is a popular sport played between two players (singles) or two teams of two players each (doubles). The game involves using a racket to hit a ball over a net into the opponent's court. \n Question: What is tennis? \n Answer: Tennis is a sport played between two players (singles) or two teams of two players each (doubles). The game involves using a racket to hit a ball over a net into the opponent's court."}]
Answer the question using the given context 
 Context: Tennis is a popular sport played between two players (singles) or two teams of two players each (doubles). The game involves using a racket to hit a ball over a net into the opponent's court. 
 Questi

In [ ]:
ques = "What is gated models?"
response = retrieve_and_generate(ques)
print(response)

I don't know the answer


In [ ]:
ques = "In a deuce situation, explain step-by-step how a player can lose the game even after gaining the advantage."
response = retrieve_and_generate(ques)
print(response)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Similarity Score: 0.8098894953727722
Retrieved Context: ```plaintext
0 points  -> Love
1 point   -> 15
2 points  -> 30
3 points  -> 40
4 points  -> Game (if leading by 2)
Deuce     -> 40-40 (must win two consecutive points to win the game)
Advantage -> If a player wins a point at deuce, they gain the advantage
```
[{'generated_text': "Answer the question using the given context \n Context: ```plaintext\n0 points  -> Love\n1 point   -> 15\n2 points  -> 30\n3 points  -> 40\n4 points  -> Game (if leading by 2)\nDeuce     -> 40-40 (must win two consecutive points to win the game)\nAdvantage -> If a player wins a point at deuce, they gain the advantage\n``` \n Question: In a deuce situation, explain step-by-step how a player can lose the game even after gaining the advantage. \n Answer: ```plaintext\nLet's say a player is at deuce. The other player is serving. The player who is serving is now at deuce. If the player who is serving wins the point, they win the game. However, if the other pla